# KRX 주가 데이터 수집 (FinanceDataReader, Colab)
한국 주식(KOSPI/KOSDAQ/KONEX) 종목 리스트와 일별 OHLCV(시가/고가/저가/종가/거래량) 데이터를 Colab에서 쉽게 불러오는 노트북입니다.

- 데이터 소스: FinanceDataReader
- 사용 방법: 아래 셀을 순서대로 실행하세요.


In [ ]:
# 필수 라이브러리 설치
!pip -q install --upgrade finance-datareader pandas matplotlib


In [ ]:
# 기본 임포트 및 유틸 함수 정의
import pandas as pd
import FinanceDataReader as fdr
from IPython.display import display

LISTING_CACHE = None

def list_krx_stocks(refresh: bool = False) -> pd.DataFrame:
    """
    KRX(유가/코스닥/코넥스) 종목 리스트를 반환합니다.
    컬럼 예시: ['Code','Name','Market','Sector','Industry']
    """
    global LISTING_CACHE
    if refresh or LISTING_CACHE is None:
        LISTING_CACHE = fdr.StockListing('KRX')
    cols = [c for c in ['Code','Name','Market','Sector','Industry'] if c in LISTING_CACHE.columns]
    return LISTING_CACHE[cols].copy() if cols else LISTING_CACHE.copy()

def resolve_code(query: str, listings: pd.DataFrame | None = None) -> dict:
    """
    6자리 종목코드 또는 한글 종목명으로 실제 코드를 해석합니다.
    여러 종목이 매치될 경우 상위 후보를 출력하고 예외를 발생합니다.
    """
    df = listings if listings is not None else list_krx_stocks()
    q = str(query).strip()
    # 코드 형태(6자리 숫자) 우선
    if q.isdigit() and len(q) == 6 and (df['Code'] == q).any():
        row = df.loc[df['Code'] == q].iloc[0]
        return {'code': row['Code'], 'name': row['Name'], 'market': row.get('Market')}
    # 정확한 이름 매치
    exact = df.loc[df['Name'] == q]
    if len(exact) == 1:
        row = exact.iloc[0]
        return {'code': row['Code'], 'name': row['Name'], 'market': row.get('Market')}
    # 포함 매치
    contains = df[df['Name'].str.contains(q, case=False, na=False)]
    if contains.empty:
        raise ValueError(f'해당 종목을 찾을 수 없습니다: {query}')
    if len(contains) > 1:
        display(contains.head(20)[[c for c in ['Code','Name','Market'] if c in contains.columns]])
        raise ValueError('여러 종목이 조회되었습니다. 6자리 코드 또는 정확한 이름을 입력하세요.')
    row = contains.iloc[0]
    return {'code': row['Code'], 'name': row['Name'], 'market': row.get('Market')}

def fetch_prices(query: str, start: str | None = None, end: str | None = None) -> pd.DataFrame:
    """
    일별 OHLCV를 조회합니다.
    query: 6자리 코드(예: '005930') 또는 종목명(예: '삼성전자')
    start/end: 'YYYY-MM-DD' 또는 None
    """
    sym = resolve_code(query)
    df = fdr.DataReader(sym['code'], start, end)
    # 메타정보를 attrs로 보관
    df.attrs.update(sym)
    return df


## 빠른 시작
- 아래 셀에서 `QUERY`, `START`, `END` 값을 설정하고 실행하세요.
- `QUERY`에는 6자리 코드 또는 한글 종목명을 넣을 수 있습니다.


In [ ]:
# 조회 파라미터 설정
QUERY = '삼성전자'   # 예: '005930' 또는 '삼성전자'
START = '2024-01-01' # 예: 'YYYY-MM-DD' 또는 None
END = None           # 예: 'YYYY-MM-DD' 또는 None

sym = resolve_code(QUERY)
print(f"종목: {sym['code']} ({sym['name']}) / 시장: {sym.get('market')}")
df = fdr.DataReader(sym['code'], START, END)
display(df.tail())


## 종목 리스트 살펴보기
- 전체 KRX 종목 리스트 로드 및 일부 미리보기
- 이름에 특정 키워드(예: '전자')가 포함된 종목 필터링 예시


In [ ]:
listings = list_krx_stocks()
print(f'전체 종목 수: {len(listings):,}')
display(listings.head(10))

# 키워드로 이름 검색(대소문자 무시)
keyword = '전자'
display(listings[listings['Name'].str.contains(keyword, case=False, na=False)].head(20))


## CSV로 저장하기
Colab 런타임의 `/content` 경로에 CSV 파일을 저장합니다.


In [ ]:
# 위의 df(최근 조회 결과)를 CSV로 저장
out_path = f"/content/{sym['code']}_{sym['name']}_ohlcv.csv"
df.to_csv(out_path, encoding='utf-8')
print('저장 완료:', out_path)


## 시각화
- 종가와 이동평균선(20, 60), 거래량을 함께 표시합니다.


In [ ]:
import matplotlib.pyplot as plt

# 이동평균선 계산
if 'Close' in df.columns:
    df['MA20'] = df['Close'].rolling(20).mean()
    df['MA60'] = df['Close'].rolling(60).mean()

fig, ax1 = plt.subplots(figsize=(12, 5))
df['Close'].plot(ax=ax1, color='tab:blue', label='Close')
if 'MA20' in df.columns: df['MA20'].plot(ax=ax1, color='tab:orange', label='MA20')
if 'MA60' in df.columns: df['MA60'].plot(ax=ax1, color='tab:green', label='MA60')
ax1.set_title(f"{sym['name']} ({sym['code']}) Price and moving average")
ax1.set_xlabel('Date')
ax1.set_ylabel('Price')
ax1.grid(alpha=0.3)
ax1.legend(loc='upper left')

# 거래량 보조축
if 'Volume' in df.columns:
    ax2 = ax1.twinx()
    ax2.bar(df.index, df['Volume'], color='lightgray', alpha=0.3, width=2, label='Volume')
    ax2.set_ylabel('Volume')
    ax2.legend(loc='upper right')

plt.show()


- MA20: 20-period simple moving average. 일봉 기준 최근 20거래일 종가의 평균으로, 대략 1개월 추세를 부드럽게 보여줍니다.
- MA60: 60-period simple moving average. 일봉 기준 최근 60거래일(약 분기)의 평균으로, 중기 추세를 나타냅니다.
- 해석: 가격이 MA 위면 상승 우위, 아래면 하락 우위로 보는 경향. MA20이 MA60을 상향 돌파하면 상승(골든크로스), 하향 돌파하면 하락(데드크로스) 신호로 해석하
기도 합니다.
- 민감도: MA20은 빠르고 민감(신호 많음, 노이즈도 많음), MA60은 느리지만 안정적(노이즈 적음, 신호 지연).
- 주의: 이동평균은 후행 지표라 횡보장에선 잦은 훼이크(휩쏘)가 발생할 수 있습니다. 데이터 주기가 시간봉/주봉이면 ‘20/60 시간/주’로 해석됩니다.
- 파이썬 계산: df['MA20'] = df['Close'].rolling(20).mean() / df['MA60'] = df['Close'].rolling(60).mean()
- 변형: 가중치 부여한 EMA 사용 예시 df['EMA20'] = df['Close'].ewm(span=20, adjust=False).mean() (반응 더 빠름).

## 해외 주식 (아마존)
- 미국 `AMZN` 티커(아마존) 일별 OHLCV 조회 및 시각화 예시입니다.
- 통화 단위는 USD입니다. 기간을 바꾸려면 `START_US`, `END_US`를 수정하세요.


In [ ]:
# 아마존(AMZN) 데이터 조회
US_TICKER = 'AMZN'
START_US = '2024-01-01'
END_US = None

us_df = fdr.DataReader(US_TICKER, START_US, END_US)
print(f'로드 완료: {US_TICKER}, 행 수={len(us_df):,}')
display(us_df.tail())


In [ ]:
# 아마존 시각화 (종가 + MA20/MA60 + 거래량)
import matplotlib.pyplot as plt

if 'Close' in us_df.columns:
    us_df['MA20'] = us_df['Close'].rolling(20).mean()
    us_df['MA60'] = us_df['Close'].rolling(60).mean()

fig, ax1 = plt.subplots(figsize=(12, 5))
us_df['Close'].plot(ax=ax1, color='tab:blue', label='Close')
if 'MA20' in us_df.columns: us_df['MA20'].plot(ax=ax1, color='tab:orange', label='MA20')
if 'MA60' in us_df.columns: us_df['MA60'].plot(ax=ax1, color='tab:green', label='MA60')
ax1.set_title('Amazon.com, Inc. (AMZN) Price and moving average')
ax1.set_xlabel('Date')
ax1.set_ylabel('Price (USD)')
ax1.grid(alpha=0.3)
ax1.legend(loc='upper left')

if 'Volume' in us_df.columns:
    ax2 = ax1.twinx()
    ax2.bar(us_df.index, us_df['Volume'], color='lightgray', alpha=0.3, width=2, label='Volume')
    ax2.set_ylabel('Volume')
    ax2.legend(loc='upper right')

plt.show()


## 국내 vs 해외 비교
- 동일 기간을 기준 100으로 정규화하여 라인 차트로 비교합니다.


In [ ]:
import matplotlib.pyplot as plt

# 국내(df)와 해외(us_df)가 준비되어 있어야 합니다.
dom = df[['Close']].rename(columns={'Close': 'Korea'}) if 'Close' in df.columns else None
us = us_df[['Close']].rename(columns={'Close': 'Amazon'}) if 'Close' in us_df.columns else None
if dom is None or us is None:
    raise RuntimeError('df 또는 us_df가 준비되어 있지 않습니다. 위의 셀들을 먼저 실행하세요.')

# 공통 기간으로 정렬/결합
merged = dom.join(us, how='inner')
if merged.empty:
    raise RuntimeError('겹치는 기간이 없습니다. START/END 범위를 조정하세요.')

# 기준 100으로 정규화
base = merged.iloc[0]
norm = merged / base * 100

ax = norm.plot(figsize=(12, 5), title='국내(Korea) vs 해외(Amazon) 기준100 비교')
ax.set_ylabel('Index (Base=100)')
ax.grid(alpha=0.3)
plt.show()


# 지수/환율 수집
한국 지수, 미국 지수, 국가별 지수, 환율을 각각 불러오는 예시입니다.


In [ ]:
# 공통 헬퍼: 심볼 후보 리스트를 순서대로 시도하여 첫 번째 성공 결과 반환
from typing import List, Tuple

def read_first_available(candidates: List[str], start=None, end=None) -> Tuple[pd.DataFrame, str]:
    last_err = None
    for sym in candidates:
        try:
            df = fdr.DataReader(sym, start, end)
            if df is not None and len(df) > 0:
                return df, sym
        except Exception as e:
            last_err = e
            continue
    raise RuntimeError(f'심볼 후보 모두 실패: {candidates} / 마지막 오류: {last_err}')

START_IDX = '2023-01-01'
END_IDX = None


## 한국 지수
- 코스피(KOSPI), 코스닥(KOSDAQ)을 불러와 비교합니다.


In [ ]:
kospi_df, kospi_sym = read_first_available(['KS11', 'KOSPI'], START_IDX, END_IDX)
kosdaq_df, kosdaq_sym = read_first_available(['KQ11', 'KOSDAQ'], START_IDX, END_IDX)
print('KOSPI 심볼:', kospi_sym, '행수:', len(kospi_df))
print('KOSDAQ 심볼:', kosdaq_sym, '행수:', len(kosdaq_df))

kr = pd.DataFrame({
    'KOSPI': kospi_df['Close'],
    'KOSDAQ': kosdaq_df['Close'],
}).dropna()
# 표로 미리보기
display(kr.tail())
base = kr.iloc[0]
kr_norm = kr / base * 100
ax = kr_norm.plot(figsize=(12,5), title='KOSPI vs KOSDAQ (기준100)')
ax.set_ylabel('Index (Base=100)'); ax.grid(alpha=0.3)
plt.show()


## 미국 지수
- S&P 500, Dow Jones, NASDAQ Composite를 불러와 비교합니다.


In [ ]:
spx_df, spx_sym = read_first_available(['US500', '^GSPC', 'GSPC'], START_IDX, END_IDX)
dji_df, dji_sym = read_first_available(['DJI', '^DJI'], START_IDX, END_IDX)
ixic_df, ixic_sym = read_first_available(['IXIC', '^IXIC', 'US100', 'NDX'], START_IDX, END_IDX)
print('S&P500 심볼:', spx_sym, '행수:', len(spx_df))
print('DJI 심볼:', dji_sym, '행수:', len(dji_df))
print('NASDAQ 심볼:', ixic_sym, '행수:', len(ixic_df))

us_idx = pd.DataFrame({
    'S&P500': spx_df['Close'],
    'DJI': dji_df['Close'],
    'NASDAQ': ixic_df['Close'],
}).dropna()
# 표로 미리보기
display(us_idx.tail())
base = us_idx.iloc[0]
us_norm = us_idx / base * 100
ax = us_norm.plot(figsize=(12,5), title='미국 지수 비교 (기준100)')
ax.set_ylabel('Index (Base=100)'); ax.grid(alpha=0.3)
plt.show()


## 국가별 지수
- 일본(니케이225), 독일(DAX), 홍콩(항셍) 등 대표 지수를 불러와 비교합니다.
- 심볼은 환경에 따라 다를 수 있어 후보를 순차 시도합니다.


In [ ]:
countries = {
    'Nikkei225': ['JP225', 'N225', '^N225'],
    'DAX': ['DE30', 'DAX', '^GDAXI'],
    'HangSeng': ['HK50', 'HSI', '^HSI'],
}
dfs = {}
for name, cands in countries.items():
    dfc, used = read_first_available(cands, START_IDX, END_IDX)
    dfs[name] = dfc['Close']
    print(f'{name}: 사용 심볼 {used}, 행수 {len(dfc)}')

cc = pd.DataFrame(dfs).dropna()
# 표로 미리보기
display(cc.tail())
base = cc.iloc[0]
cc_norm = cc / base * 100
ax = cc_norm.plot(figsize=(12,5), title='국가별 지수 비교 (기준100)')
ax.set_ylabel('Index (Base=100)'); ax.grid(alpha=0.3)
plt.show()


## 환율
- USD/KRW, JPY/KRW, EUR/KRW 환율을 불러와 비교합니다. (종가 기준)


In [ ]:
fx_pairs = {
    'USD/KRW': ['USD/KRW'],
    'JPY/KRW': ['JPY/KRW'],
    'EUR/KRW': ['EUR/KRW'],
}
fx_closes = {}
for name, cands in fx_pairs.items():
    dffx, used = read_first_available(cands, START_IDX, END_IDX)
    fx_closes[name] = dffx['Close']
    print(f'{name}: 사용 심볼 {used}, 행수 {len(dffx)}')

fx = pd.DataFrame(fx_closes).dropna()
# 표로 미리보기
display(fx.tail())
# 환율은 절대수준 비교도 가능하지만, 변동 비교를 위해 기준 100 정규화 예시
base = fx.iloc[0]
fx_norm = fx / base * 100
ax = fx_norm.plot(figsize=(12,5), title='환율 비교 (기준100)')
ax.set_ylabel('Index (Base=100)'); ax.grid(alpha=0.3)
plt.show()


## 암호화폐 (비트코인)
- BTC를 특정 통화쌍으로 조회하고 표와 시각화를 제공합니다.
- 예: `BTC/USD`, `BTC/KRW`


In [ ]:
BTC_PAIR = 'BTC/USD'   # 'BTC/KRW' 로 변경 가능
START_CRYPTO = '2024-01-01'
END_CRYPTO = None

btc = fdr.DataReader(BTC_PAIR, START_CRYPTO, END_CRYPTO)
print(f'로드 완료: {BTC_PAIR}, 행 수={len(btc):,}')
display(btc.tail())

# 시각화 (종가 + MA20/MA60 + 거래량 보조축)
import matplotlib.pyplot as plt
if 'Close' in btc.columns:
    btc['MA20'] = btc['Close'].rolling(20).mean()
    btc['MA60'] = btc['Close'].rolling(60).mean()
fig, ax1 = plt.subplots(figsize=(12,5))
btc['Close'].plot(ax=ax1, label='Close', color='tab:blue')
if 'MA20' in btc.columns: btc['MA20'].plot(ax=ax1, label='MA20', color='tab:orange')
if 'MA60' in btc.columns: btc['MA60'].plot(ax=ax1, label='MA60', color='tab:green')
ax1.set_title(f'Bitcoin ({BTC_PAIR}) Close & MAs')
ax1.set_xlabel('Date'); ax1.set_ylabel('Price')
ax1.grid(alpha=0.3); ax1.legend(loc='upper left')
if 'Volume' in btc.columns:
    ax2 = ax1.twinx()
    ax2.bar(btc.index, btc['Volume'], color='lightgray', alpha=0.3, width=2, label='Volume')
    ax2.set_ylabel('Volume')
    ax2.legend(loc='upper right')
plt.show()


## 암호화폐 (여러 종목)
- ETH, XRP, SOL 등 주요 종목을 불러와 표로 확인 후 정규화 비교 그래프를 출력합니다.
- 통화쌍 후보를 순서대로 시도하여 사용 가능한 심볼을 자동 선택합니다.


In [ ]:
crypto_candidates = {
    'ETH': ['ETH/USD', 'ETH/KRW'],
    'XRP': ['XRP/USD', 'XRP/KRW'],
    'SOL': ['SOL/USD', 'SOL/KRW'],
}
crypto_closes = {}
for name, cands in crypto_candidates.items():
    dfc, used = read_first_available(cands, START_CRYPTO, END_CRYPTO)
    crypto_closes[name] = dfc['Close']
    print(f'{name}: 사용 심볼 {used}, 행수 {len(dfc)}')

crypto_df = pd.DataFrame(crypto_closes).dropna()
display(crypto_df.tail())
base = crypto_df.iloc[0]
crypto_norm = crypto_df / base * 100
ax = crypto_norm.plot(figsize=(12,5), title='암호화폐 비교 (기준100)')
ax.set_ylabel('Index (Base=100)'); ax.grid(alpha=0.3)
plt.show()


## 통합 시각화
- 위에서 만든 모든(또는 실행된) 데이터 시리즈를 한 그래프에 색상으로 구분해 표시합니다.
- 서로 다른 자산군이므로 값의 절대 비교 대신 기준100 정규화로 상대 비교합니다.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def norm_to_100(s: pd.Series):
    s = s.dropna()
    if s.empty:
        return None
    base = s.iloc[0]
    if base == 0:
        return None
    return s / base * 100

series_dict = {}  # label -> normalized series

# 국내 단일 종목
try:
    if 'df' in globals() and 'Close' in df.columns:
        label = f"KOR Stock: {sym.get('name', '')}({sym.get('code','')})" if 'sym' in globals() else 'KOR Stock'
        ns = norm_to_100(df['Close'])
        if ns is not None: series_dict[label] = ns
except Exception:
    pass

# 아마존
try:
    if 'us_df' in globals() and 'Close' in us_df.columns:
        ns = norm_to_100(us_df['Close'])
        if ns is not None: series_dict['AMZN'] = ns
except Exception:
    pass

# 한국/미국/국가별 지수 표에서 가져오기 (있을 때만)
for name in ['kr', 'us_idx', 'cc']:
    try:
        if name in globals():
            df_idx = globals()[name]
            for col in df_idx.columns:
                ns = norm_to_100(df_idx[col])
                if ns is not None:
                    series_dict[col] = ns
    except Exception:
        pass

# 환율
try:
    if 'fx' in globals():
        for col in fx.columns:
            ns = norm_to_100(fx[col])
            if ns is not None: series_dict[col] = ns
except Exception:
    pass

# 비트코인 단일
try:
    if 'btc' in globals() and 'Close' in btc.columns:
        ns = norm_to_100(btc['Close'])
        if ns is not None: series_dict[f'BTC ({BTC_PAIR})'] = ns
except Exception:
    pass

# 기타 암호화폐
try:
    if 'crypto_df' in globals():
        for col in crypto_df.columns:
            ns = norm_to_100(crypto_df[col])
            if ns is not None: series_dict[f'CRYPTO {col}'] = ns
except Exception:
    pass

# 통합 데이터프레임 구성 (outer join)
if not series_dict:
    raise RuntimeError('통합할 시리즈가 없습니다. 위의 섹션들을 먼저 실행하세요.')

unified = pd.concat(series_dict, axis=1)  # columns are labels
unified = unified.dropna(how='all')
display(unified.tail())

# 색상 지정(tab20 팔레트에서 순차 할당)
labels = list(unified.columns)
colors = [cm.get_cmap('tab20')(i % 20) for i in range(len(labels))]
color_map = dict(zip(labels, colors))

fig, ax = plt.subplots(figsize=(14, 6))
for lab in labels:
    ax.plot(unified.index, unified[lab], label=lab, color=color_map[lab], linewidth=1.5)
ax.set_title('통합 비교 (기준100)')
ax.set_ylabel('Index (Base=100)')
ax.grid(alpha=0.3)
ax.legend(ncol=2, fontsize=9)
plt.show()


## Gradio 대시보드
- 선택한 시리즈를 기준100 정규화 여부와 기간을 조절해 한 그래프로 선명하게(Plotly) 표시합니다.


In [ ]:
# Gradio/Plotly 설치 (Colab에선 보통 Plotly 기본 제공)
!pip -q install gradio plotly

import pandas as pd
import numpy as np
import gradio as gr
import plotly.graph_objects as go
import plotly.express as px

def collect_series_store():
    store = {}  # label -> pd.Series (raw Close)
    # 국내 개별
    try:
        if 'df' in globals() and 'Close' in df.columns:
            label = f"KOR Stock: {sym.get('name','')}({sym.get('code','')})" if 'sym' in globals() else 'KOR Stock'
            store[label] = df['Close']
    except Exception: pass
    # AMZN
    try:
        if 'us_df' in globals() and 'Close' in us_df.columns:
            store['AMZN'] = us_df['Close']
    except Exception: pass
    # 지수 표들
    for name in ['kr', 'us_idx', 'cc']:
        try:
            if name in globals():
                df_idx = globals()[name]
                for col in df_idx.columns:
                    store[col] = df_idx[col]
        except Exception: pass
    # 환율
    try:
        if 'fx' in globals():
            for col in fx.columns:
                store[col] = fx[col]
    except Exception: pass
    # BTC 단일
    try:
        if 'btc' in globals() and 'Close' in btc.columns:
            store[f'BTC ({globals().get('BTC_PAIR', 'BTC')} )'] = btc['Close']
    except Exception: pass
    # 기타 크립토
    try:
        if 'crypto_df' in globals():
            for col in crypto_df.columns:
                store[f'CRYPTO {col}'] = crypto_df[col]
    except Exception: pass
    return store

SERIES_STORE = collect_series_store()
CHOICES = list(SERIES_STORE.keys())
if not CHOICES:
    print('주의: 선택할 시리즈가 없습니다. 위 섹션들을 먼저 실행하세요.')

def build_colors(n, palette='Plotly'):
    if palette == 'Viridis':
        seq = px.colors.sequential.Viridis
    elif palette == 'Turbo':
        seq = px.colors.sequential.Turbo
    elif palette == 'Set2':
        seq = px.colors.qualitative.Set2
    else:
        seq = px.colors.qualitative.Plotly
    # 반복하여 n개 확보
    colors = [seq[i % len(seq)] for i in range(n)]
    return colors

def normalize_series(s):
    s = s.dropna()
    if s.empty: return s
    base = s.iloc[0]
    if base == 0: return s
    return s / base * 100

def plot_unified(selected, start, end, normalize, palette):
    if not selected:
        return go.Figure(), pd.DataFrame()
    # 시리즈 정리
    data = {}
    for lab in selected:
        if lab in SERIES_STORE:
            s = SERIES_STORE[lab]
            if start:
                s = s[s.index >= start]
            if end:
                s = s[s.index <= end]
            if normalize:
                s = normalize_series(s)
            data[lab] = s
    if not data:
        return go.Figure(), pd.DataFrame()
    # 공통 인덱스로 결합 (outer) 및 표 미리보기
    merged = pd.concat(data, axis=1)
    # Plotly 시각화
    colors = build_colors(len(data), palette)
    fig = go.Figure()
    for (lab, s), color in zip(data.items(), colors):
        fig.add_trace(go.Scatter(x=s.index, y=s.values, mode='lines', name=lab, line=dict(color=color, width=2)))
    fig.update_layout(title='선택 시리즈 통합 비교', yaxis_title='Index (Base=100)' if normalize else 'Value', hovermode='x unified', template='plotly_white', height=600, legend=dict(orientation='h'))
    return fig, merged.tail(20)


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown('### 선택한 시리즈를 한눈에 비교 (Plotly)')
    series = gr.CheckboxGroup(choices=CHOICES, value=CHOICES[:min(4, len(CHOICES))] if CHOICES else [], label='표시할 시리즈')
    with gr.Row():
        start = gr.Textbox(label='시작일 YYYY-MM-DD', value='2024-01-01')
        end = gr.Textbox(label='종료일 YYYY-MM-DD', value='')
        normalize = gr.Checkbox(label='기준100 정규화', value=True)
        palette = gr.Dropdown(['Plotly','Viridis','Turbo','Set2'], value='Plotly', label='색상 팔레트')
    plot = gr.Plot(label='그래프')
    table = gr.Dataframe(label='표 미리보기', wrap=True)
    btn = gr.Button('그리기')
    btn.click(plot_unified, inputs=[series, start, end, normalize, palette], outputs=[plot, table])

demo.launch(share=False)
